In [2]:
import torchio as tio
import torch
import os
import matplotlib.pyplot as plt
import numpy as np
import argparse
from models import *
from utilities import *

/Users/huayinluo/Documents/code/gapjncsegmentation/gapvenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
def get_z_y_x(file_name, pattern):
    """ get z, y, x from file name (uses basename of file_name)"""
    file_name = os.path.basename(file_name)
    match = re.match(pattern, file_name)
    if match:
        if len(match.groups()) != 3:
            return None
        z, y, x = match.groups()
        return int(z), int(y), int(x)
    else:
        return None

In [33]:
gt_dir="/Volumes/LaCie/zhenlab/gjsegmentation/gj_test_datasets/100_110_3x512x512/ground_truth"
img_dir="/Volumes/LaCie/zhenlab/gjsegmentation/gj_test_datasets/100_110_3x512x512/original"
save_dir="/Volumes/LaCie/filtered_100_110_3x512x512"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(os.path.join(save_dir, "original", "train")):
    os.makedirs(os.path.join(save_dir, "original", "train"), exist_ok=True)
if not os.path.exists(os.path.join(save_dir, "original", "valid")):
    os.makedirs(os.path.join(save_dir, "original", "valid"), exist_ok=True)
if not os.path.exists(os.path.join(save_dir, "ground_truth", "train")):
    os.makedirs(os.path.join(save_dir, "ground_truth", "train"), exist_ok=True)
if not os.path.exists(os.path.join(save_dir, "ground_truth", "valid")):
    os.makedirs(os.path.join(save_dir, "ground_truth", "valid"), exist_ok=True)

In [35]:
img_paths = os.listdir(img_dir)
gt_paths = os.listdir(gt_dir)
print(len(img_paths))
print(img_paths[:10])
print(gt_paths[:10])

1271
['z0_y0_x0.npy', 'z0_y0_x1024.npy', 'z0_y0_x1536.npy', 'z0_y0_x2048.npy', 'z0_y0_x2560.npy', 'z0_y0_x3072.npy', 'z0_y0_x3584.npy', 'z0_y0_x4096.npy', 'z0_y0_x4608.npy', 'z0_y0_x512.npy']
['z0_y0_x0.npy', 'z0_y0_x1024.npy', 'z0_y0_x1536.npy', 'z0_y0_x2048.npy', 'z0_y0_x2560.npy', 'z0_y0_x3072.npy', 'z0_y0_x3584.npy', 'z0_y0_x4096.npy', 'z0_y0_x4608.npy', 'z0_y0_x512.npy']


In [38]:
import shutil
imgs=[]
gts=[]
num_saved = 0
for i in range(len(img_paths)):
    img_fp = img_paths[i]
    gt_fp = gt_paths[i]
    try:
        img = np.load(os.path.join(img_dir, img_fp))
        gt = np.load(os.path.join(gt_dir, gt_fp))
    except:
        continue
    suffix = os.path.splitext(img_fp)[0]
    print(f"Saved {suffix} | {i}/{len(img_paths)} | num saved {num_saved}", end="\r")
    depth = gt.shape[0]
    has_empty = False
    for k in range(depth):
        if len(np.unique(gt[k])) < 2:
            has_empty = True
    if not has_empty:
        np.save(os.path.join(save_dir, "original", "train", f"{suffix}.npy"), img)
        np.save(os.path.join(save_dir, "ground_truth", "train", f"{suffix}.npy"), gt)
        num_saved += 1

In [43]:
img_paths = os.listdir(os.path.join(save_dir, "original", "train"))
gt_paths = os.listdir(os.path.join(save_dir, "ground_truth", "train"))
print(len(img_paths))
print(img_paths[:10])
print(gt_paths[:10])

316
['z0_y1536_x3584.npy', 'z0_y1536_x4096.npy', 'z0_y1536_x4608.npy', 'z0_y2048_x2560.npy', 'z0_y2048_x3072.npy', 'z0_y2048_x3584.npy', 'z0_y2048_x4096.npy', 'z0_y2048_x4608.npy', 'z0_y2048_x5120.npy', 'z0_y2048_x6144.npy']
['z0_y1536_x3584.npy', 'z0_y1536_x4096.npy', 'z0_y1536_x4608.npy', 'z0_y2048_x2560.npy', 'z0_y2048_x3072.npy', 'z0_y2048_x3584.npy', 'z0_y2048_x4096.npy', 'z0_y2048_x4608.npy', 'z0_y2048_x5120.npy', 'z0_y2048_x6144.npy']


In [45]:
num_imgs=len(img_paths)
num_valid = int(0.1*num_imgs)
indices = np.arange(num_imgs)
np.random.shuffle(indices)
valid_indices = indices[:num_valid]
print(len(valid_indices))
print(valid_indices[:10])

126
[293  65  81 170 125 314 203 212 215 236]


In [47]:
save_dir="/Volumes/LaCie/filtered_100_110_3x512x512_backup"

In [46]:
for i in range(len(valid_indices)):
    idx=valid_indices[i]
    print(f"moved {i}/{num_valid}")
    img_fp = img_paths[idx]
    gt_fp = gt_paths[idx]
    os.rename(os.path.join(save_dir, "original","train", img_fp), os.path.join(save_dir, "original","valid", img_fp))
    os.rename(os.path.join(save_dir, "ground_truth","train", gt_fp), os.path.join(save_dir, "ground_truth","valid", gt_fp))

moved 0/126
moved 1/126
moved 2/126
moved 3/126
moved 4/126
moved 5/126
moved 6/126
moved 7/126
moved 8/126
moved 9/126
moved 10/126
moved 11/126
moved 12/126
moved 13/126
moved 14/126
moved 15/126
moved 16/126
moved 17/126
moved 18/126
moved 19/126
moved 20/126
moved 21/126
moved 22/126
moved 23/126
moved 24/126
moved 25/126
moved 26/126
moved 27/126
moved 28/126
moved 29/126
moved 30/126
moved 31/126
moved 32/126
moved 33/126
moved 34/126
moved 35/126
moved 36/126
moved 37/126
moved 38/126
moved 39/126
moved 40/126
moved 41/126
moved 42/126
moved 43/126
moved 44/126
moved 45/126
moved 46/126
moved 47/126
moved 48/126
moved 49/126
moved 50/126
moved 51/126
moved 52/126
moved 53/126
moved 54/126
moved 55/126
moved 56/126
moved 57/126
moved 58/126
moved 59/126
moved 60/126
moved 61/126
moved 62/126
moved 63/126
moved 64/126
moved 65/126
moved 66/126
moved 67/126
moved 68/126
moved 69/126
moved 70/126
moved 71/126
moved 72/126
moved 73/126
moved 74/126
moved 75/126
moved 76/126
moved 77/

In [ ]:


# -- args
start_x = 
start_y = 
start_z = 
ending_depth = 
ending_height = 
ending_width = 
subvol_depth = 
subvol_height = 
subvol_width = 
mask_dir= 
img_dir= 
save_dir=
if not os.path.exists(os.path.join(save_dir, "visualize")): 
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(os.path.join(save_dir, "original"), exist_ok=True)
    os.makedirs(os.path.join(save_dir, "ground_truth"), exist_ok=True)
    os.makedirs(os.path.join(save_dir, "pred"), exist_ok=True)
    os.makedirs(os.path.join(save_dir, "visualize"), exist_ok=True)
    
# -- get imgs & mask fp
img_files = os.listdir(img_dir)
mask_files = os.listdir(mask_dir)
mask_pattern=r"sem_dauer_2_gj_gt_s(\d+).png"
img_pattern=r"SEM_dauer_2_em_s(\d+).png"
img_files = [os.path.join(img_dir, f) for f in img_files if f.endswith(".png")]
mask_files = [os.path.join(mask_dir, f) for f in mask_files if f.endswith(".png")]

# -- load model
fp=
model = UNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model, optimizer, epoch, loss, batch_size, lr, focal_loss_weights = load_checkpoint(model, optimizer, fp)
model = model.eval()
print("model loaded")

In [ ]:

# -- get masks:
i=0
for z in range(start_z, ending_depth):
    tmp_img = get_img_by_z(z, img_files, img_pattern)
    tmp_mask = get_img_by_z(z, mask_files, mask_pattern)
    if i == 0:
        h, w = tmp_img.shape[0], tmp_img.shape[1]
        full_volume_img = np.zeros((ending_depth-start_z, h, w))
        full_volume_mask = np.zeros((ending_depth-start_z, h, w))
        print("Created full img & mask", full_volume_img.shape, full_volume_mask.shape)
    full_volume_img[i] = tmp_img
    full_volume_mask[i] = tmp_mask
    i+=1
    print(f"done {z} z-slice")
print("full volume shape:", full_volume_img.shape, full_volume_mask.shape)

start_z = 0
# ending_depth = full_volume_img.shape[0]
if args.use_full_volume:
    ending_height = full_volume_img.shape[1]
    ending_width = full_volume_img.shape[2]
print("subvolume shape:", subvol_depth, subvol_height, subvol_width)
print("ending:", ending_depth, ending_height, ending_width)

step_z = args.step_z
step_y = args.step_y
step_x = args.step_x
while start_z < ending_depth:
    end_z = start_z + subvol_depth
    start_y = args.start_y
    while start_y < ending_height:
        end_y = start_y + subvol_height 
        start_x = args.start_x
        while start_x < ending_width:
            end_x = start_x + subvol_width
            sub_volume_img = full_volume_img[start_z:end_z, start_y:end_y, start_x:end_x]
            sub_volume_mask = full_volume_mask[start_z:end_z, start_y:end_y, start_x:end_x] # with confidence levels
            sub_vol_depth, sub_vol_height, sub_vol_width = sub_volume_img.shape
            image = torch.tensor(sub_volume_img).float().unsqueeze(0)
            if (sub_vol_height < subvol_height) or (sub_vol_width < subvol_width) or (sub_vol_depth < subvol_depth):
                image = tio.CropOrPad((subvol_depth, subvol_height, subvol_width))(image)
                
            try:
                image = tio.ZNormalization()(image)
                print(image.shape)
                intermed_pred, sub_volume_pred = model(image)
                binary_pred = torch.argmax(sub_volume_pred[0], dim=0) # (depth, height, width)
                np.save(os.path.join(save_dir, "original", f"z{start_z}_y{start_y}_x{start_x}.npy"), sub_volume_img)
                np.save(os.path.join(save_dir, "ground_truth", f"z{start_z}_y{start_y}_x{start_x}.npy"), sub_volume_mask)
                np.save(os.path.join(save_dir, "pred", f"z{start_z}_y{start_y}_x{start_x}.npy"), sub_volume_pred.detach().cpu())            
                fig, ax = plt.subplots(3, subvol_depth, figsize=(15,5), num=1)
                visualize_3d_slice(sub_volume_img, ax[0], "Image")
                visualize_3d_slice(sub_volume_mask, ax[1], "Mask")
                visualize_3d_slice(binary_pred, ax[2], "Pred")
                plt.savefig(os.path.join(save_dir, "visualize", f"z{start_z}_y{start_y}_x{start_x}.png"))
                plt.close("all")
                print(f"Saved z{start_z}-{end_z} y{start_y}-{end_y} x{start_x}-{end_x} subvolume")
            except:
                print(f"Skipping z{start_z}-{end_z} y{start_y}-{end_y} x{start_x}-{end_x}")
            start_x = start_x + step_x
        start_y = start_y + step_y
    start_z = start_z + step_z